### Phase 1: Ising Model

$$H = \sum_{i=1}^{N} a_i X_i + \sum_{i=1}^{N} \sum_{j=1}^{i-1} J_{ij} Z_i Z_j
$$

In [ ]:
import numpy as np
from qat.core import Observable, Term

def ising(N):
    np.random.seed(123)  

    terms = []

    # Generate random coefficients for the transverse field term (X)
    a_coefficients = np.random.random(N)
    for i in range(N):
        term = Term(coefficient=a_coefficients[i], pauli_op="X", qbits=[i])
        terms.append(term)

    # Generate random coefficients for the interaction term (ZZ)
    J_coefficients = np.random.random((N, N))
    for i in range(N):
        for j in range(i):
            if i != j:  # avoid duplicate terms
                term = Term(coefficient=J_coefficients[i, j], pauli_op="ZZ", qbits=[i, j])
                terms.append(term)
    ising = Observable(N, pauli_terms=terms, constant_coeff=0.0)
    return ising

In [ ]:
model = ising(4) 
print("Transverse Ising Model:", model)

In [ ]:
print(model.nbqbits)

In [ ]:
print(model.to_matrix)

Based on the other result of initial state that has been choosen for the ising model creat the ansazt that apply the value k for two case where it generate the value 0010 and 0011 

In [ ]:
from numpy import binary_repr
from qat.lang import Program, X
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin
import scipy.optimize

In [ ]:
from qat.lang.AQASM import Program, QRoutine, RY, CNOT, RX, Z, H, RZ, I
from qat.core import Observable, Term, Circuit
from qat.lang.AQASM.gates import Gate
import matplotlib as mpl
import numpy as np
from typing import Optional, List
import warnings


In [ ]:
def HEA_slow(
    nqbits: int,
    theta: List[float],
    k: int,
    n_cycles: int = 1,
    rotation_gates: List[Gate] = None,
    entangling_gate: Gate = CNOT,
    
) -> Circuit: #linear entanglement
    """
    This Hardware Efficient Ansatz has the reference from "Nonia Vaquero Sabater et al. Simulating molecules 
    with variational quantum eigensolvers. 2022" -Figure 6 -Link 
    "https://uvadoc.uva.es/bitstream/handle/10324/57885/TFM-G1748.pdf?sequence=1"

    Args:
        nqbits (int): Number of qubits of the circuit.
        n_cycles (int): Number of layers.
        rotation_gates (List[Gate]): Parametrized rotation gates to include around the entangling gate. Defaults to :math:`RY`. Must
            be of arity 1.
        entangling_gate (Gate): The 2-qubit entangler. Must be of arity 2. Defaults to :math:`CNOT`.
    """

    if rotation_gates is None:
        rotation_gates = [RZ]

    n_rotations = len(rotation_gates)

    prog = Program()
    reg = prog.qalloc(nqbits)
    theta = [prog.new_var(float, rf"\theta_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
    # the angle parameters were randomly sampled from as index variable
    #theta = [np.random.uniform(0, 2*np.pi) for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
    print("Hello")
    ind_theta = 0

    ket_value = binary_repr(k)
    padded_ket_value = ket_value.zfill(nqbits)
    list_ket_value = [int(c) for c in padded_ket_value]

        # Apply X gates based on the binary representation of the current value
    for j in range(min(nqbits, len(list_ket_value))):
        if list_ket_value[j] == 1:
            prog.apply(X, reg[j])
        if list_ket_value[j] == 0:
            prog.apply(I, reg[j])

    
    for i in range(nqbits):

        for rot in rotation_gates:

            prog.apply(rot(theta[ind_theta]), reg[i])
            ind_theta += 1
    
    for k in range(n_cycles):


        for i in range(nqbits - 1):
            prog.apply(CNOT, reg[i], reg[i+1])
            
        for i in range(nqbits):
            for rot in rotation_gates:
                            
                prog.apply(rot(theta[ind_theta]), reg[i])
                ind_theta += 1

    return prog.to_circ()

In [ ]:
theta_0 = np.random.random(size=model.nbqbits * 2)




In [ ]:
from numpy import binary_repr
from qat.lang import Program, X
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin
import scipy.optimize
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from qat.qpus import get_default_qpu
from qat.core import Batch, Job

from qat.fermion.transforms import transform_to_jw_basis
from qat.fermion.hamiltonians import make_embedded_model
from qat.fermion.circuits import make_shallow_circ, make_ldca_circ

from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry import MolecularHamiltonian, MoleculeInfo
from qat.fermion.trotterisation import make_trotterisation_routine
from qat.fermion.chemistry.ucc import construct_ucc_ansatz, guess_init_params, get_hf_ket, get_cluster_ops
from qat.qpus import get_default_qpu

from qat.plugins import ScipyMinimizePlugin, MultipleLaunchesAnalyzer

In [ ]:
import scipy.optimize


In [ ]:
def energy_Linear_theta_s(nqbits,theta, k, n_cycles, rotational_gate, entangling_gate):
    print("Hello")
    circ_Linear_s = HEA_slow(model.nbqbits,theta, k, n_cycles, [RX,RZ], CNOT)
    print("Hello")
    jobE_s = circ_Linear_s.to_job(job_type="OBS",observable=model)
    qpu = get_default_qpu()
    resE_s = qpu.submit(jobE_s)
    energy_Linear_theta_s = resE_s.value
    print("Energy value for this iterative is", energy_Linear_theta_s)

    return energy_Linear_theta_s

In [ ]:
theta_0 = np.random.random(size=model.nbqbits * 2)
nbqbits =4
energy_Linear_theta_s(nbqbits,theta, 0, 3, [RX,RZ], CNOT),

opt_result_Linear_s = scipy.optimize.minimize(
            lambda theta: energy_Linear_theta_s(nbqbits,theta, 0, 3, [RX,RZ], CNOT),
            x0= theta_0,
            method= "SLSQP",
            tol=1e-2,
            options={"maxiter": 10, "disp": True})
print(opt_result_Linear_s)